## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [14]:
import os
from dotenv import load_dotenv

def get_credentials():
    return {
        "url": "https://us-south.ml.cloud.ibm.com",
        "apikey": os.getenv('API_KEY')
    }


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [3]:
model_id = "ibm/granite-13b-chat-v2"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [4]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1.05
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [7]:
project_id = os.getenv('PROJECT_ID')
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [8]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [23]:
prompt_input = """<|system|>
You are Granite, an AI language model developed by IBM, integrated into a suite of four applications: ConnectX (for meetings), DocX (for document management), CalX (for calendar events), and MailX (for emails). Your primary role is to assist users by generating structured JSON responses only that automate tasks across these applications, based on the user's prompt.
When processing a user prompt, carefully analyze which applications are relevant and respond strictly using the following JSON format:
ConnectX: Return 1 if the prompt requires generating a meeting link, and 0 if it does not.
DocX: Return 1 if the prompt involves creating or editing a document, and 0 if it does not.
CalX: ONLY if the prompt involves a date-specific event, return a key-value (date-description) pair where the date is the key (formatted as DD-MM-YY), and a concise description (no more than 10 words) is the value. Otherwise return calX: 0. Always use the DD-MM-YY format for dates. Add multiple entries for mulitple dates or days.
MailX: If the prompt includes the need to generate an email, return the subject (max 12 words) and the body (max 300 words) as key-value pairs. If no email is needed, return mailX: 0. If the user explicitly states that no email is required, always ensure to set mailX: 0.
Key Rules:
Do not provide any additional commentary, explanations, or text outside of the JSON structure under any circumstances. Always adhere strictly to the format and provide a fully structured and accurate JSON response based on the user prompt.
Never respond with any text other than the required JSON structure.
Never include comments or explanations alongside the JSON.
Always return the correct JSON structure based on the user’s input, ensuring compliance with the format.
The response must always be concise, precise, and limited to the exact JSON output.
Avoid calX events is not mentioned specifically.
Example JSON response:
{
  "connectX": 0,
  "docX": 0,
  "calX": {
    "00-00-00": "Sign NDA"},
  "mailX": {
    "sub": "Reminder: NDA Signing Deadline – 00th February",
    "body": "I hope this message finds you well. I’m writing to remind you that the deadline for signing the Non-Disclosure Agreement (NDA) is approaching on 00-00"
  }
}

"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [24]:
question = input("Question: ")
formattedQuestion = f"""<|user|>
{question}
<|assistant|>
"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"{generated_response}")


Here is a randomly generated JSON response that creates a random event on a random date with a random meeting and mail:

```json
{
  "connectX": 0,
  "docX": 0,
  "calX": {
    "randomDate": "2023-02-27T12:00:00Z": "Random Meeting"
  },
  "mailX": {
    "subject": "Random Meeting Reminder - 27th February",
    "body": "Hey there! I'm just dropping you a quick reminder about our upcoming Random Meeting scheduled for 00:00 on 2023-02-27. I believe it will be quite an interesting discussion. See you there! 😊"
  }
}
```

In this example, the `calX` field creates a random event on a random date in the format `YYYY-MM-DD` (`2023-02-27T12:00:00Z`). The `mailX` field then generates a random meeting reminder email with a specific subject and body. Note that the `connectX` and `docX` fields remain 0 since no meeting link generation or document creation is required in this scenario.
